In [2]:
import torch
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
import matplotlib.pyplot as plt
import copy


In [4]:
# Load Images
def load_image(image_path, max_size=400):
    image = Image.open(image_path).convert('RGB')
    
    # Resize the image
    size = max_size if max(image.size) > max_size else max(image.size)
    transform = transforms.Compose([
        transforms.Resize((size, size)),
        transforms.ToTensor()
    ])
    
    image = transform(image)[:3, :, :].unsqueeze(0)
    return image.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

In [6]:
# Display Image
def show_image(tensor):
    image = tensor.clone().detach().cpu().squeeze(0)
    image = transforms.ToPILImage()(image)
    plt.imshow(image)
    plt.axis('off')
    plt.show()

In [8]:
# Load Pre-trained VGG19 Model
class VGG19Features(torch.nn.Module):
    def __init__(self):
        super(VGG19Features, self).__init__()
        self.model = models.vgg19(pretrained=True).features[:29].eval()
        for param in self.model.parameters():
            param.requires_grad = False
    
    def forward(self, x):
        features = {}
        layers = {
            '1': 'conv1_1', '6': 'conv2_1', '11': 'conv3_1', '20': 'conv4_1', '29': 'conv5_1'
        }
        
        for name, layer in self.model._modules.items():
            x = layer(x)
            if name in layers:
                features[layers[name]] = x
        return features

In [10]:
# Compute Content Loss
def content_loss(target_features, generated_features):
    return torch.mean((generated_features - target_features) ** 2)

# Compute Style Loss
def gram_matrix(tensor):
    _, d, h, w = tensor.size()
    tensor = tensor.view(d, h * w)
    return torch.mm(tensor, tensor.t())

def style_loss(style_features, generated_features):
    loss = 0
    for layer in style_features:
        gram_s = gram_matrix(style_features[layer])
        gram_g = gram_matrix(generated_features[layer])
        loss += torch.mean((gram_s - gram_g) ** 2)
    return loss

In [ ]:
# Neural Style Transfer
def neural_style_transfer(content_path, style_path, steps=500, alpha=1e4, beta=1e8):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    content = load_image(content_path).to(device)
    style = load_image(style_path).to(device)
    
    model = VGG19Features().to(device)
    target = content.clone().requires_grad_(True).to(device)
    optimizer = optim.Adam([target], lr=0.003)
    
    content_features = model(content)
    style_features = model(style)
    
    for step in range(steps):
        optimizer.zero_grad()
        generated_features = model(target)
        c_loss = content_loss(content_features['conv4_1'], generated_features['conv4_1'])
        s_loss = style_loss(style_features, generated_features)
        total_loss = alpha * c_loss + beta * s_loss
        total_loss.backward()
        optimizer.step()
        
        if step % 100 == 0:
            print(f"Step {step}/{steps}, Loss: {total_loss.item():.4f}")
    
    show_image(target)
    return target

# Example Usage
styled_image = neural_style_transfer(r"C:\Users\poojan\OneDrive\Desktop\CODETECH_IT_SOLUTION\sample1.jpg", r"C:\Users\poojan\OneDrive\Desktop\CODETECH_IT_SOLUTION\sample2.jpg")